In [1]:
!pip install roboflow
!pip install ultralytics

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="CMpIGYPcrZeCpVDmVeYV")
project = rf.workspace("karel-cornelis-q2qqg").project("aicook-lcv4d")
version = project.version(4)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
from ultralytics import YOLO
import os

# Load the YOLOv8s model
model = YOLO("yolov8s.pt")  # Load the pretrained YOLOv8s model

# Train the model
results = model.train(
    data=os.path.join(dataset.location, 'data.yaml'),
    epochs=50, # Increased epochs for more training
    batch=32,
    imgsz=640,
    save=True,
    project='runs/detect',
    name='food_detection_optimized', # Changed name to reflect optimization
    lr0=0.001 # Reduced initial learning rate for fine-tuning
)

Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/aicook-4/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=food_detection_optimized2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0

In [ ]:
# Check if running in Colab or local environment
import os
import glob

# Find the latest training run
train_dirs = glob.glob("runs/detect/food_detection*")
if train_dirs:
    latest_train = max(train_dirs, key=os.path.getctime)
    model_path = os.path.join(latest_train, "weights", "best.pt")
else:
    model_path = "runs/detect/train/weights/best.pt"  # fallback

model = YOLO(model_path)  # load the trained model

# Validate the model
metrics = model.val()
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")

In [ ]:
import os
import sys

if 'google.colab' in sys.modules:
    # Google Colab environment
    from google.colab import files
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]
else:
    # Local environment - specify image path directly
    image_path = input("Enter the path to your image file: ")
    if not os.path.exists(image_path):
        print("Image file not found!")
        image_path = None

In [ ]:
from ultralytics import YOLO
import os
import glob

# Check if running in Colab or local environment
import os
import glob

# Find the latest training run
train_dirs = glob.glob("runs/detect/food_detection*")
if train_dirs:
    latest_train = max(train_dirs, key=os.path.getctime)
    model_path = os.path.join(latest_train, "weights", "best.pt")
else:
    model_path = "runs/detect/food_detection/weights/best.pt" #fallback

model = YOLO(model_path)  # load the trained model

if image_path and os.path.exists(image_path):
    results = model.predict(
        image_path,
        save=True,
        visualize=False,
        conf=0.5,
        save_txt=True,
        project='runs/detect',
        name='predictions'
    )
    print("Prediction completed! Results saved in runs/detect/predictions/")
else:
    print("No valid image path provided.")

In [ ]:
import sys
import os

if 'google.colab' in sys.modules:
    # Google Colab - mount drive and save
    from google.colab import drive
    drive.mount('/content/drive')
    save_dir = '/content/drive/MyDrive/project-1'
    os.makedirs(save_dir, exist_ok=True) # Create the directory if it doesn't exist
    save_path = os.path.join(save_dir, 'yolov8s_trained_model.pt')
else:
    # Local environment - save to project directory
    save_path = 'models/yolov8s_food_detection_trained.pt'
    os.makedirs('models', exist_ok=True)

model.save(save_path)
print(f"Model saved to: {save_path}")

In [ ]:
print("\n=== Training Summary ===")
print(f"Model saved at: {save_path}")
print(f"Best mAP50-95: {metrics.box.map:.4f}")
print(f"Best mAP50: {metrics.box.map50:.4f}")
print("Training completed successfully!")

In [ ]:
import os
import sys

if 'google.colab' in sys.modules:
    # Google Colab environment
    from google.colab import files
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]
else:
    # Local environment - specify image path directly
    image_path = input("Enter the path to your image file: ")
    if not os.path.exists(image_path):
        print("Image file not found!")
        image_path = None